In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install autogluon
!pip install mxnet~=1.9

In [ ]:
import pandas as pd
import numpy as np
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from autogluon.common import space

In [ ]:
path = '/content/drive/MyDrive/DACON/jeju_price/'

# For TG V2

In [ ]:
train_df = pd.read_csv(path + 'train.csv')[['ID','timestamp','supply(kg)', 'price(원/kg)']]
train_df['item_id'] = train_df.ID.str[0:6]
train_df

,ID,timestamp,supply(kg),price(원/kg),item_id
0,TG_A_J_20190101,2019-01-01,0.0,0.0,TG_A_J
1,TG_A_J_20190102,2019-01-02,0.0,0.0,TG_A_J
2,TG_A_J_20190103,2019-01-03,60601.0,1728.0,TG_A_J
3,TG_A_J_20190104,2019-01-04,25000.0,1408.0,TG_A_J
4,TG_A_J_20190105,2019-01-05,32352.0,1250.0,TG_A_J
...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,452440.0,468.0,RD_F_J
59393,RD_F_J_20230228,2023-02-28,421980.0,531.0,RD_F_J
59394,RD_F_J_20230301,2023-03-01,382980.0,574.0,RD_F_J
59395,RD_F_J_20230302,2023-03-02,477220.0,523.0,RD_F_J


In [ ]:
train_df = train_df[(train_df["item_id"] != "BC_B_S") & (train_df["item_id"] != "BC_C_S") & (train_df["item_id"] != "CB_A_S") & (train_df["item_id"] != "CR_D_S") & (train_df["item_id"] != "CR_E_S") & (train_df["item_id"] != "RD_C_S")]
train_df = train_df.reset_index(drop = True)
train_df.drop(columns=['ID'], inplace=True)

In [ ]:
train_df["timestamp"] = pd.to_datetime(train_df["timestamp"])

In [ ]:
for df in [train_df]:
    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month
    df['day'] = df['timestamp'].dt.day
    df['weekday'] = df['timestamp'].dt.weekday
    df['week'] = df['timestamp'].dt.isocalendar().week
    df['quarter'] = df['timestamp'].dt.quarter
    df['days_in_month'] = df['timestamp'].dt.days_in_month
    df['is_month_start'] = df['timestamp'].dt.is_month_start
    df['is_month_end'] = df['timestamp'].dt.is_month_end
    df['is_quarter_start'] = df['timestamp'].dt.is_quarter_start
    df['is_quarter_end'] = df['timestamp'].dt.is_quarter_end
    df['is_year_start'] = df['timestamp'].dt.is_year_start
    df['is_year_end'] = df['timestamp'].dt.is_year_end
    df['is_leap_year'] = df['timestamp'].dt.is_leap_year
    df['dayofyear'] = df['timestamp'].dt.dayofyear

In [ ]:
seed_number = 5
seed_list = list(range(41, 41 + seed_number))
pred_all = []
for seed in seed_list:
    data = TimeSeriesDataFrame(train_df)
    predictor = TimeSeriesPredictor(
        prediction_length=28,
        target="price(원/kg)",
        eval_metric="RMSE",
    )
    # seed 고정
    predictor.fit(data, random_seed=seed)
    predictor.refit_full()
    pred = predictor.predict(data, random_seed=seed, )
    pred_all.append(pred)

In [ ]:
submission = pd.read_csv(path + 'sample_submission.csv')

submission["item_id"] = submission["ID"].str[0:6]
submission["TIME"] = submission["ID"].str[10:]

submission = submission[(submission["item_id"] != "BC_B_S") & (submission["item_id"] != "BC_C_S") & (submission["item_id"] != "CB_A_S") & (submission["item_id"] != "CR_D_S") & (submission["item_id"] != "CR_E_S") & (submission["item_id"] != "RD_C_S")]
submission = submission.reset_index(drop = True)

for i in range(len(pred_all)):
    # Ensure that 'mean' is a valid column name in pred[i]
    if 'mean' in pred_all[i].columns:
        submission['answer'] += pred_all[i].reset_index()['mean']
    else:
        print(f"DataFrame at index {i} does not contain 'mean' column")

submission['answer'] = submission['answer'] / len(pred_all)
# submission['answer'] = pred.reset_index()['mean']
sunday_ids_march_2023 = [f"_202303{day:02}" for day in [5, 12, 19, 26]]
submission.loc[submission['answer'] < 0.0, 'answer'] = 0.0


# Set the 'answer' to 0 for those Sundays using ID column
for sunday_id in sunday_ids_march_2023:
    submission.loc[submission['ID'].str.contains(sunday_id), 'answer'] = 0
#submission.to_csv('FOR_TGv2_owm.csv', index=False)
submission.to_csv(path + 'preprov1.csv',index=False)

In [ ]:
sb = pd.read_csv(path + 'autogloun_hype_final.csv')
sb["item_id"] = sb["ID"].str[0:6]
sb["TIME"] = sb["ID"].str[10:]
sb.to_csv(path + 'preprov2.csv',index=False)
sb

In [ ]:
# 'original_order' 열 추가
sb['original_order'] = range(len(sb))

# 인덱스 설정
submission.set_index(['item_id', 'TIME'], inplace=True)
sb.set_index(['item_id', 'TIME'], inplace=True)

# 공통 인덱스 찾기
common_indices = sb.index.intersection(submission.index)

# 'original_order' 열 보존하면서 sb 업데이트
sb_update = sb.loc[common_indices].copy()
sb_update.update(submission.loc[common_indices])
sb.loc[common_indices] = sb_update

# 인덱스 리셋 및 원래 순서로 정렬
sb.reset_index(inplace=True)
sb.sort_values(by='original_order', inplace=True)

# 불필요한 'original_order' 열 제거
sb.drop(columns=['original_order'], inplace=True)
sb.drop(['item_id','TIME'],axis=1,inplace=True)

# CSV 파일로 저장
sb.to_csv(path + 'TG_TOALL_RE1_seed.csv', index=False)
sb